In [1]:
import os
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
import re
from bs4 import BeautifulSoup
import requests
from datetime import datetime

In [2]:
round_ = '18'
url = f'https://www.sportsbet.com.au/betting/australian-rules/afl/round-{round_}'
url

'https://www.sportsbet.com.au/betting/australian-rules/afl/round-18'

In [3]:
day = datetime.now().strftime("%A").lower()
day

'friday'

In [4]:
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

In [5]:
tmp = soup.find_all('script')

In [6]:
game_urls = []
for i in tmp[:9]:
    try:
        url_find = re.compile(r'https://www.sportsbet.com.au/betting/australian-rules/afl/(.*?)\-\d{7}')
        game_url = url_find.search(i.text)
        game_urls.append(game_url.group())
    except:
        continue
game_urls

['https://www.sportsbet.com.au/betting/australian-rules/afl/melbourne-v-brisbane-7470681',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/collingwood-v-fremantle-7470682',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/gold-coast-v-st-kilda-7470684',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/carlton-v-port-adelaide-7470685',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/geelong-v-essendon-7470687',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/adelaide-v-greater-western-sydney-7470688',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/north-melbourne-v-hawthorn-7470689',
 'https://www.sportsbet.com.au/betting/australian-rules/afl/west-coast-v-richmond-7470691']

In [7]:
r = re.compile(r'\/(\w*\-?\w*\-?\w*)\-v\-(\w*\-?\w*\-?\w*)\-')
game_list = []
for game in game_urls:
    game_list.append(r.findall(game))

In [8]:
game_list

[[('melbourne', 'brisbane')],
 [('collingwood', 'fremantle')],
 [('gold-coast', 'st-kilda')],
 [('carlton', 'port-adelaide')],
 [('geelong', 'essendon')],
 [('adelaide', 'greater-western-sydney')],
 [('north-melbourne', 'hawthorn')],
 [('west-coast', 'richmond')]]

In [9]:
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.set_window_size(700,800)

[WDM] - Downloading: 100%|█████████████████| 8.29M/8.29M [00:00<00:00, 13.6MB/s]


In [10]:
disposals = ['To Get 15 or More Disposals',
            'To Get 20 or More Disposals',
            'To Get 25 or More Disposals',
            'To Get 30 or More Disposals',]
output = pd.DataFrame({},columns = {'teams','round','Player', 'disposals', 'odds'})

In [ ]:
for game, game_url in zip(game_list, game_urls):
    driver.get(game_url)
    try:
        first_click = driver.find_element_by_xpath('(//*[@class="size14_f7opyze medium_f1wf24vo"])')
        first_click.click()
        
        for i in range(1,7):
            if driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze medium_f1wf24vo"])[{i}]').text == 'Disposal Markets':
                disposal_market = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze medium_f1wf24vo"])[{i}]')
        
        
        disposal_market.click()
        time.sleep(3)
        
        for i in range(5,50):
            try:
                tmp = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze bold_f1au7gae"])[{i}]').text
            except:
                continue
            if tmp in disposals:

                dis = driver.find_element_by_xpath(f'(//*[@class="size14_f7opyze bold_f1au7gae"])[{i}]')

                dis.click()

                player = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg outcomeDetailsFirst_f162fqwy"])').text.split('\n')
                print(player)
                new_row = {'Player': player[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':tmp,'odds':player[-1]}
                output.loc[len(output)] = new_row

                for j in range(1,50):
                    try:

                        tmp1 = driver.find_element_by_xpath(f'(//*[@class="outcomeContainer_f1wc7xgg"])[{j}]').text.split('\n')
                        print(tmp1)
                        new_row = {'Player': tmp1[0].lower(), 'teams': f'{game[0][0]} v {game[0][1]}', 'round':f'R{round_}', 'disposals':tmp,'odds':tmp1[-1]}
                        output.loc[len(output)] = new_row
                    except:
                        continue
                
                dis.click()
                time.sleep(2)
            
    except:
        continue

['Lachie Hunter', '1.06']
['Trent Rivers', '1.11']
['Tom Sparrow', '1.15']
['Ed Langdon', '1.18']
['James Jordon', '1.18']
['Steven May', '1.18']
['Max Gawn', '1.22']
['Jake Bowey', '1.32']
['Jake Lever', '1.49']
['Alex Neal-Bullen', '1.68']
['Taj Woewodin', '1.98']
['Judd McVee', '2.07']
['Kysaiah Pickett', '2.25']
['Charlie Spargo', '2.95']
['Jacob van Rooyen', '3.80']
['Dayne Zorko', '1.25']
['Zac Bailey', '1.26']
['Conor McKenna', '1.32']
['Keidean Coleman', '1.35']
['Jarrod Berry', '1.40']
['Darcy Wilmot', '1.49']
['Joe Daniher', '1.50']
['Cam Rayner', '1.55']
['Harris Andrews', '1.69']
['Jaspa Fletcher', '1.93']
['Oscar McInerney', '2.30']
['Brandon Starcevich', '2.55']
['Jack Gunston', '2.55']
['Lincoln McCarthy', '2.95']
['Angus Brayshaw', '1.14']
['Jack Viney', '1.17']
['Lachie Hunter', '1.41']
['Christian Salem', '1.57']
['Trent Rivers', '1.72']
['Tom Sparrow', '1.89']
['Ed Langdon', '1.93']
['Steven May', '1.94']
['James Jordon', '1.95']
['Max Gawn', '2.30']
['Jake Bowey', '

['Mason Wood', 'Last 5: 21-24-24-18-16', '1.08']
['Rowan Marshall', 'Last 5: 30-26-13-17-16', '1.08']
['Zak Jones', 'Last 5: 20-14-17-4-18', '1.10']
['Brad Hill', 'Last 5: 16-25-31-19-19', '1.13']
['Callum Wilkie', 'Last 5: 23-16-15-17-23', '1.22']
['Marcus Windhager', 'Last 5: 19-20-12-20-15', '1.30']
['Ryan Byrnes', 'Last 5: 18-12-7-10-19', '1.34']
['Jade Gresham', 'Last 5: 24-21-11-16-18', '1.35']
['Liam Stocker', 'Last 5: 19-9-18-13-13', '1.42']
['Jimmy Webster', 'Last 5: 20-15-15-13-19', '1.43']
['Dougal Howard', 'Last 5: 17-18-12-14-13', '1.76']
['Mitchito Owens', 'Last 5: N/A', '2.05']
['Mattaes Phillipou', 'Last 5: 15-11-12-18-10', '2.35']
['Jack Peris', 'Last 5: N/A', '2.90']
['Touk Miller', 'Last 5: 20-13-26-26-32', '1.16']
['Sam Flanders', 'Last 5: 24-27-27-13-7', '1.35']
['Darcy MacPherson', 'Last 5: 15-22-25-22-12', '1.41']
['Matt Rowell', 'Last 5: 21-20-20-21-15', '1.52']
['Rory Atkins', 'Last 5: 26-14-7-21-17', '1.54']
['Wil Powell', 'Last 5: 14-27-19-18-17', '1.67']
['B

['Sam Docherty', 'Last 5: 28-26-32-27-26', '2.85']
['Blake Acres', 'Last 5: 23-22-17-21-18', '6.25']
['Nic Newman', 'Last 5: 23-17-19-6-19', '10.50']
['Adam Saad', 'Last 5: 18-24-18-22-14', '12.00']
['Zak Butters', 'Last 5: 22-25-23-31-26', '2.80']
['Connor Rozee', 'Last 5: 28-23-25-23-29', '3.50']
['Ollie Wines', 'Last 5: 26-13-28-18-24', '4.90']
['Dan Houston', 'Last 5: 22-32-31-17-20', '5.00']
['Travis Boak', 'Last 5: 20-8-13-26-12', '7.00']
['Gryan Miers', 'Last 5: 32-27-15-13-20', '1.09']
['Isaac Smith', 'Last 5: 18-17-20-22-24', '1.09']
['Max Holmes', 'Last 5: 26-8-17-24-13', '1.18']
['Jack Bowes', 'Last 5: 25-3-12-5-3', '1.19']
['Tanner Bruhn', 'Last 5: 22-18-19-12-15', '1.22']
['Tom Atkins', 'Last 5: 25-18-19-12-19', '1.23']
['Mark Blicavs', 'Last 5: 18-16-19-13-17', '1.29']
['Zach Guthrie', 'Last 5: 21-10-12-26-16', '1.48']
['Jeremy Cameron', 'Last 5: 0-10-21-17-10', '1.49']
['Brad Close', 'Last 5: 16-16-24-9-18', '1.54']
['Tom Hawkins', 'Last 5: 14-9-13-10-10', '2.30']
['Rhys

['Jack Buckley', 'Last 5: 16-12-12-10-13', '8.25']
['Jesse Hogan', 'Last 5: 15-7-17-9-19', '8.25']
['Rory Laird', 'Last 5: 31-24-31-34-22', '1.25']
['Jordan Dawson', 'Last 5: 28-28-35-32-30', '1.28']
['Ben Keays', 'Last 5: 20-29-29-29-25', '1.97']
['Mitch Hinge', 'Last 5: 27-26-23-18-16', '3.25']
['Brodie Smith', 'Last 5: 21-24-19-18-26', '3.75']
['Rory Sloane', 'Last 5: 12-30-19-19-17', '4.90']
['Chayce Jones', 'Last 5: 16-16-23-21-17', '6.25']
['Wayne Milera', 'Last 5: 16-18-18-21-20', '6.75']
['Lachlan Sholl', 'Last 5: 16-21-11-17-15', '8.75']
['Izak Rankine', 'Last 5: 11-17-21-20-13', '11.00']
['Lachlan Murphy', 'Last 5: 4-18-7-16-16', '11.00']
['Josh Rachele', 'Last 5: 15-23-11-15-12', '12.00']
['Josh Worrell', 'Last 5: 16-13-14-18-19', '14.00']
['Stephen Coniglio', 'Last 5: 30-30-31-25-23', '1.30']
['Josh Kelly', 'Last 5: 6-26-33-24-27', '1.51']
['Lachie Whitfield', 'Last 5: 21-22-24-33-31', '1.95']
['Callan Ward', 'Last 5: 19-19-16-26-32', '2.45']
['Harry Himmelberg', 'Last 5: N

['Nathan Broad', 'Last 5: 8-21-17-19-18', '1.36']
['Noah Balta', 'Last 5: 15-24-17-13-17', '1.36']
['Jack Graham', 'Last 5: 19-11-20-21-9', '1.37']
['Trent Cotchin', 'Last 5: 15-14-29-28-14', '1.50']
['Ivan Soldo', 'Last 5: 11-10-6-5-7', '1.89']
['Kamdyn McIntosh', 'Last 5: 15-10-16-11-19', '1.93']
['Marlion Pickett', 'Last 5: 6-9-10-7-17', '2.20']


In [ ]:
output

In [ ]:
output['home'] = output['teams'].apply(lambda x: x.split(' v ')[0])
output['away'] = output['teams'].apply(lambda x: x.split(' v ')[1])

In [ ]:
team_mapping = {'richmond':'RCH','geelong':'GEE',
                'north-melbourne':'NTH','port-adelaide':'POR',
                'gold-coast':'GCS','melbourne':'MEL',
                'greater-western-sydney':'GWS','hawthorn':'HAW',
                'carlton':'CAR','brisbane':'BRL',
                'collingwood':'COL','adelaide':'ADE',
                'fremantle':'FRE','western-bulldogs':'WBD',
                'sydney':'SYD','west-coast':'WCE',
                'st-kilda':'STK','essendon':'ESS'}

In [ ]:
for key in team_mapping.keys():
     output.loc[output['home']==key, 'home'] = team_mapping[key]
for key in team_mapping.keys():
    output.loc[output['away']==key, 'away'] = team_mapping[key]     

In [ ]:
output

In [ ]:
output.to_csv(f'/Users/caseyh/Desktop/footyfootyfooty/sportsbet_odds/r{round_}/disposal_markets_{day}.csv', index = False)